In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
# gemini_uri_generator.py
from langchain_google_genai import ChatGoogleGenerativeAI
import json

# 1. setup the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# 2. the reusable function
def generate_uri_with_gemini(query: str, resource_spec: dict) -> str:
    prompt = f"""
You are a URI generator for REST APIs.
Convert the user request into a GET URI using only the information in the given resource spec.

Rules:
- Use the base_path from the spec as the starting point of the URI.
- Use query parameters exactly as defined in query_params.map.
- Use sensible defaults from query_params.defaults when missing.
- Normalize 'vegetarian' → 'veg', 'non veg' → 'non-veg'.
- If user says "around N", set price.min = N*0.75 and price.max = N*1.25
- If user says "cheap", set max_price=400 (if not given).
- Return **only** one line: the final URI starting with "GET".
- No explanations or JSON.

Resource spec:
{json.dumps(resource_spec, indent=2)}

User query: {query}
Return only the URI.
"""

    response = llm.invoke(prompt)
    return response.content.strip()

# 3. example resource spec
MENU_SPEC = {
    "base_path": "/api/v1/menu",
    "query_params": {
        "map": {
            "category": "category",
            "type": "type",
            "price.min": "min_price",
            "price.max": "max_price",
            "rating.min": "min_rating",
            "search": "search"
        },
        "defaults": {"page": 1, "limit": 20}
    },
    "notes": "type synonyms: vegetarian→veg, non veg→non-veg; rating 0–5"
}

# 4. demo
if __name__ == "__main__":
    tests = [
        "non veg pizzas under 500 best rated",
        "veg momo around 200",
        "show me burgers below 350",
        "find pizza rated 4+",
        "cheap veg pizza",
        "what's the price of pizza"
    ]

    for q in tests:
        print(f"\nUser: {q}")
        print("URI:", generate_uri_with_gemini(q, MENU_SPEC))



User: non veg pizzas under 500 best rated
URI: GET /api/v1/menu?type=non-veg&category=pizzas&max_price=500&min_rating=5&page=1&limit=20

User: veg momo around 200
URI: GET /api/v1/menu?type=veg&search=momo&min_price=150&max_price=250&page=1&limit=20

User: show me burgers below 350
URI: GET /api/v1/menu?category=burgers&max_price=350&page=1&limit=20

User: find pizza rated 4+
URI: GET /api/v1/menu?search=pizza&min_rating=4&page=1&limit=20

User: cheap veg pizza
URI: GET /api/v1/menu?max_price=400&type=veg&search=pizza&page=1&limit=20

User: what's the price of pizza
URI: GET /api/v1/menu?search=pizza


In [2]:
# router_agent_minimal.py
# pip install langchain-google-genai
# export GOOGLE_API_KEY=...

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

SYSTEM_PROMPT = """
You are the Router for a restaurant assistant. Classify the user's message into exactly one intent:
- menu: questions about food/menu items, categories, veg/non-veg, ingredients, allergens, prices, recommendations, ratings, combos.
- about: questions about the restaurant itself: name, location/address, opening/closing hours, contact/phone, story/vision/mission, ambience.
- faqs: policies/amenities: parking, Wi-Fi, payment methods, pet/kid friendliness, wheelchair access, smoking area, delivery partners, refunds/returns, general facilities.
- escalate: requests to place/modify/cancel orders, make reservations/table bookings, payment/transaction issues, urgent complaints requiring staff.
- none: greetings, thanks, small talk, jokes, or generic conversation you can handle directly without tools.

Rules:
- Prefer escalate if the user asks to "book", "reserve", "order", "pay", "cancel", or similar operational requests.
- Prefer menu when the focus is food details, even if price/rating is mentioned.
- Prefer about for hours, address, contact, or the restaurant’s story.
- Prefer faqs for policies and facilities.
- Use none for chit-chat (hi/hello/thanks/ok/lol/good night) or general praise.

Return ONLY JSON in one line:
{"intent":"menu|about|faqs|escalate|none","confidence":0.0-1.0,"rationale":"<short reason>"}
Keep rationale under 12 words.
"""

def classify_intent_minimal(user_text: str) -> str:
    """
    Minimal version:
    - Sends system prompt + user text to the LLM
    - Returns the LLM's raw response (a one-line JSON string per the prompt)
    - No parsing, no guards, no exception handling
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"User: {user_text}\nReturn only the JSON."},
    ]
    resp = llm.invoke(messages)
    return resp.content.strip()

# demo
if __name__ == "__main__":
    tests = [
        "hi there",
        "do you have veg pizzas under 500?",
        "what time do you open tomorrow?",
        "is there parking and wifi?",
        "can you book a table for 7pm?",
        "tell me about your founder's story",
    ]
    for t in tests:
        print(t, "->", classify_intent_minimal(t))


hi there -> {"intent":"none","confidence":1.0,"rationale":"User greeting, no specific intent or tool needed."}
do you have veg pizzas under 500? -> {"intent":"menu","confidence":1.0,"rationale":"User asks about specific menu item (veg pizzas) and price."}
what time do you open tomorrow? -> {"intent":"about","confidence":1.0,"rationale":"User asks about opening hours, which is restaurant information."}
is there parking and wifi? -> {"intent":"faqs","confidence":1.0,"rationale":"User asks about facilities like parking and Wi-Fi."}
can you book a table for 7pm? -> {"intent":"escalate","confidence":1.0,"rationale":"User wants to book a table, which is an operational request."}
tell me about your founder's story -> {"intent":"about","confidence":1.0,"rationale":"User asks about the restaurant's founder's story."}
